In [4]:
# Total sleep time이 정리되어 있는 excel 파일이 없는 경우 실행 가능한 프로그램입니다.
# 단, hypoxic burden을 계산하고자 하는 모든 PSG들의 "Polysomnography Report" pdf 파일이 필요합니다.
# ModuleNotFoundError 발생하면 오른쪽 위 "Python 3.11.2" 을 클릭 -> Select Another Kernel -> Python Environments... -> base (Python 3.9.7) 클릭
# ***** 주의 ***** 프로그램으로 돌리고 싶은 SpO2, Events, Reports 파일이 "OneDrive" 파일에 있는지 아닌지 확인!
# OneDrive 파일에 있다면, 바로 밑 5줄로 실행
SpO2_folder_path = 'C:/Users/smcsl/OneDrive/바탕 화면/PSG_study/Brain_age/Hypoxic_burden/Raw_data/6-SpO2/'
Events_folder_path = 'C:/Users/smcsl/OneDrive/바탕 화면/PSG_study/Brain_age/Hypoxic_burden/Raw_data/6-Events/'
Reports_folder_path = 'C:/Users/smcsl/OneDrive/바탕 화면/PSG_study/Brain_age/Hypoxic_burden/Raw_data/6-Reports/'
Result_path = 'C:/Users/smcsl/OneDrive/바탕 화면/PSG_study/Brain_age/Hypoxic_burden/'
Result_name = 'Hypoxic_burden_pdf_6'

# OneDrive 파일에 없다면, 위 5줄 앞에 # 하나씩 붙이고, 아래 5줄 앞에 # 지우기
#SpO2_folder_path = 'C:/Users/smcsl/Desktop/PSG_study/Brain_age/Hypoxic_burden/Raw_data/6-SpO2/'
#Events_folder_path = 'C:/Users/smcsl/Desktop/PSG_study/Brain_age/Hypoxic_burden/Raw_data/6-Events/'
#Reports_folder_path = 'C:/Users/smcsl/Desktop/PSG_study/Brain_age/Hypoxic_burden/Raw_data/6-Reports/'
#Result_path = 'C:/Users/smcsl/Desktop/PSG_study/Brain_age/Hypoxic_burden/'
#Result_name = 'Hypoxic_burden_pdf_6'

if '\\' in SpO2_folder_path:
    raise SystemError('SpO2_folder_path 에 \ 있습니다. / 으로 바꿔주세요')
if '/' != SpO2_folder_path[-1]:
    raise SystemError('SpO2_folder_path 마지막에 / 붙여주세요')
if '\\' in Events_folder_path:
    raise SystemError('Events_folder_path 에 \ 있습니다. / 으로 바꿔주세요') 
if '/' != Events_folder_path[-1]:
    raise SystemError('Events_folder_path 마지막에 / 붙여주세요')
if '\\' in Reports_folder_path:
    raise SystemError('Reports_folder_path 에 \ 있습니다. / 으로 바꿔주세요')  
if '/' != Reports_folder_path[-1]:
    raise SystemError('Reports_folder_path 마지막에 / 붙여주세요')
if '\\' in Result_path:
    raise SystemError('Result_path 에 \ 있습니다. / 으로 바꿔주세요')  
if '/' != Result_path[-1]:
    raise SystemError('Result_path 마지막에 / 붙여주세요')
    
import PyPDF2
import pandas as pd
import numpy as np
import os, sys
dir_path = os.path.dirname(os.path.realpath('tools'))
sys.path.append(os.path.abspath(os.path.join(dir_path, os.pardir)))
import re
from tqdm import tqdm
import warnings
import importlib
import WHI
importlib.reload(WHI)
warnings.filterwarnings("ignore")

def find_file_by_PSG_no(folder, PSG_no):
    txt_re = re.compile(PSG_no.lower())
    try:
        return [file for file in folder if len(txt_re.findall(file.lower())) != 0][0]
    except:
        return False

SpO2_folder = os.listdir(SpO2_folder_path)
Events_folder = os.listdir(Events_folder_path)
Reports_folder = os.listdir(Reports_folder_path)

for file in SpO2_folder:
    if ',' not in file:
        raise SystemError(file, ' 파일 이름에 PSG number 근처로 쉼표(,) 확인 부탁드립니다. 파일 이름 형식은 메뉴얼 참고 부탁드립니다.')
for file in Events_folder:
    if '-' not in file:
        raise SystemError(file, ' 파일 이름에 PSG number 근처로 띄어쓰기 또는 짝대기(-) 확인 부탁드립니다. 파일 이름 형식은 메뉴얼 참고 부탁드립니다.')
for file in Reports_folder:
    if '-' not in file:
        raise SystemError(file, ' 파일 이름에 PSG number 근처로 짝대기(-) 확인 부탁드립니다. 파일 이름 형식은 메뉴얼 참고 부탁드립니다.')

SpO2_PSG_no_list = [file.split(',')[0] for file in SpO2_folder]
Events_PSG_no_list = [file.split(' ')[1].split('-')[0] for file in Events_folder]
Reports_PSG_no_list = [file.split('-')[0] for file in Reports_folder]

PSG_no_list = list(set(SpO2_PSG_no_list) & set(Events_PSG_no_list) & set(Reports_PSG_no_list))

hypoxic_burden_df = pd.DataFrame(columns = ['Total sleep time (h)', 'Apnea-specific hypoxic burden (%min/h)', 'Hypopnea-specific hypoxic burden (%min/h)', \
    'Total hypoxic burden (%min/h)', 'Weighted hypoxemia index (upper = 90%)'], index = PSG_no_list)
    
miss_SpO2 = list(set(SpO2_PSG_no_list) - set(PSG_no_list))
miss_Events = list(set(Events_PSG_no_list) - set(PSG_no_list))
miss_Report = list(set(Reports_PSG_no_list) - set(PSG_no_list))

if len(miss_SpO2):
    print('다음이 포함된 SpO2 파일들은 이름 형식이 잘못되어 hypoxic burden 계산이 되지 않습니다.')
    print(miss_SpO2)
    print('수정하기 위해서는 상단의 정지 표시의 "Interrupt"를 눌러주시고 파일 이름 형식 수정 후 다시 실행하십시오')
    print('-------------------------------------------------------------------------------------------------------------------------------------------')   
if len(miss_Events):
    print('다음이 포함된 Event 파일들은 이름 형식이 잘못되어 hypoxic burden 계산이 되지 않습니다.')
    print(miss_Events)
    print('수정하기 위해서는 상단의 정지 표시의 "Interrupt"를 눌러주시고 파일 이름 형식 수정 후 다시 실행하십시오')
    print('-------------------------------------------------------------------------------------------------------------------------------------------')
if len(miss_Report):
    print('다음이 포함된 Report 파일들을 찾을 수가 없어, hypoxic burden 계산이 되지 않습니다.')
    print(miss_Report)
    print('수정하기 위해서는 상단의 정지 표시의 "Interrupt"를 눌러주시고 파일 이름 형식 수정 후 다시 실행하십시오')
    print('-------------------------------------------------------------------------------------------------------------------------------------------')

for PSG_no in tqdm(PSG_no_list):
    Reports_file = find_file_by_PSG_no(Reports_folder, PSG_no)
    Event_file = find_file_by_PSG_no(Events_folder, PSG_no)
    SpO2_file = find_file_by_PSG_no(SpO2_folder, PSG_no)
    if Reports_file and Event_file and SpO2_file:
        SpO2_path = SpO2_folder_path + SpO2_file
        Resp_event_path = Events_folder_path + Event_file
        Reports_path = Reports_folder_path + Reports_file
        
        pdfFileObj = open(Reports_path, 'rb')
        tst = float(PyPDF2.PdfFileReader(pdfFileObj).getPage(0).extractText().split('minutes')[5])/60

        psg = WHI.PSG(PSG_no, tst, SpO2_path, Resp_event_path)

        hypoxic_burden_df.loc[PSG_no, 'Total sleep time (h)'] = tst
        hypoxic_burden_df.loc[PSG_no, ['Apnea-specific hypoxic burden (%min/h)', 'Hypopnea-specific hypoxic burden (%min/h)', \
            'Total hypoxic burden (%min/h)']] = psg.hypoxic_burden()
        hypoxic_burden_df.loc[PSG_no, 'Weighted hypoxemia index (upper = 90%)'] = psg.calculate_whi()
    else:
        raise SystemError('Missing Report or SpO2 or Event file of ' + PSG_no)

hypoxic_burden_df.index.name = 'PSG_no'
hypoxic_burden_df.to_excel(Result_path + Result_name + '.xlsx')



 11%|█         | 1/9 [00:05<00:42,  5.33s/it]

PE250487: 1492 times down 90.0


 11%|█         | 1/9 [01:25<11:22, 85.27s/it]


KeyboardInterrupt: 